In [ ]:


!pip install langchain langchain-google-genai duckduckgo-search
import time
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool
from langchain.memory import ConversationBufferMemory
from duckduckgo_search import DDGS

# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key="YOUR_KEY_HERE"  # Replace with your key
)

# Define tools
@tool
def calculator(query: str) -> str:
    """Evaluates a mathematical expression and returns the result as a string."""
    try:
        return str(eval(query))
    except:
        return "Invalid expression"

@tool
def web_search(query: str) -> str:
    """Searches the web for information using DuckDuckGo and returns a list of titles and URLs."""
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=3)
    return str([r["title"] + ": " + r["href"] for r in results])

@tool
def file_reader(filename: str) -> str:
    """Reads and returns the content of a text file."""
    if not os.path.exists(filename):
        return f"File '{filename}' does not exist."
    try:
        with open(filename, 'r') as file:
            return file.read()
    except Exception as e:
        return f"Error reading file: {str(e)}"

tools = [calculator, web_search, file_reader]

# Initialize memory to cache responses
memory = ConversationBufferMemory()

# Initialize agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

# Test the agent (with delay to avoid quota issues)
if __name__ == "__main__":
    # Create example.txt if it doesn't exist
    if not os.path.exists("example.txt"):
        with open("example.txt", "w") as f:
            f.write("Hello, this is a test file!\nIt contains some text for the file_reader tool to read.")

    queries = [
        "What is 5 + 3 * 2?",
        "Search the web for Python tutorials",
        "Read the content of example.txt"
    ]
    for query in queries:
        response = agent.run(query)
        print(f"Query: {query}\nResponse: {response}\n")
        time.sleep(5)  # Avoid hitting API quota




